In [29]:
from langchain_community.document_loaders import JSONLoader

In [33]:
import json
from pathlib import Path
from pprint import pprint


file_path='pics/Andhra Pradesh_schemes.json'
data = json.loads(Path(file_path).read_text())
data= data[0:69]


In [34]:
from langchain.schema import Document

# Prepare the data (using 'eligibility', 'benefits', and 'schemeName' as an example)
chunked_data = []

for item in data:
    # Combine fields (eligibility, benefits, etc.) into one string, and include headings
    combined_text = ""

    # Add schemeName as a heading
    if 'schemeName' in item:
        combined_text += f"Scheme Name: {item['schemeName']}\n\n"

    # Add schemeCategory as a heading
    if 'schemeCategory' in item:
        combined_text += f"Scheme Category: {', '.join(item['schemeCategory'])}\n\n"

    if 'benefits' in item and item['benefits']:
        combined_text += "Benefits:\n" + "\n".join(item['benefits']) + "\n\n"
    # Add eligibility as a heading if it exists
    if 'eligibility' in item and item['eligibility']:
        combined_text += "Eligibility:\n" + "\n".join(item['eligibility']) + "\n\n"

    if 'documentsRequired' in item and item['documentsRequired']:
        combined_text += "Documents Required:\n" + item['documentsRequired'] + "\n\n"
    document = Document(page_content=combined_text)  # Only page_content, no metadata

    # Now append this document to chunked_data list
    chunked_data.append(document)

# Now chunked_data contains the documents in the required format (without metadata)


In [35]:
chunked_data[4]

Document(metadata={}, page_content="Scheme Name: Dr. YSR Aarogyasri Health Insurance Scheme\n\nScheme Category: Health & Wellness\n\nBenefits:\nHospital Care: When hospitalisation is needed, the system covers in-patient treatment for all of the aforementioned illnesses.\nOut-Patient Care: These treatments are provided as part of the scheme's cashless services by health camps and network hospitals.\nCashless Services: When registered family members and beneficiaries demand it, an annual coverage of ₹5,00,000 per family is provided.\nPre-Existing Disease Coverage: This plan has an unusual characteristic in that it covers ailments that the recipient was already suffering from before enrolling in the scheme.\nFollow-Up: The Dr. YSR Aarogyasri program also pays for post-hospitalization operations that can be rather expensive on their own.\nAffordable Family Health Insurance: Family health insurance allows the complete family to use the covered amount without the requirement for separate pol

In [36]:
len(chunked_data)

69

In [37]:
from langchain_cohere import CohereEmbeddings
cohere_api_key="p8eOQWcrY8direXVjNhSy7xLcrOe4mnJ4XbMm85X"
embeddings = CohereEmbeddings(
    model="embed-english-v3.0",
    cohere_api_key=cohere_api_key
)

In [38]:
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(chunked_data, embeddings)

In [39]:
db

In [40]:
db.save_local("./vector_database")
